<a href="https://colab.research.google.com/github/NULabTMN/homework-2-AkshatSavaliya/blob/main/PageRank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PageRank

In this assignment, you will compute PageRank on a collection of 469,235 web sites using the iterative version of the PageRank algorithm described in class for sparse graphs (NOT the power method with explicit matrix multiplication).

Consider the following directed graph:

![A directed link graph](https://ccs.neu.edu/home/dasmith/courses/cs6200/pagerank.jpg)

We can represent this graph as a collection of nodes, here, ordered pairs of node index and node name:

In [3]:
small_nodes = [(0, 'A'),
              (1, 'B'),
              (2, 'C'),
              (3, 'D'),
              (4, 'E'),
              (5, 'F'),
               (6, 'G'),
               (7,'H')]

and a collection of directed links, i.e., ordered pairs from source to target:

In [4]:
small_edges = [
  (0, 1),
  (0, 2),
  (0, 5),
  (1, 2),
  (1, 3),
  (1, 4),
  (1, 5),
  (2, 3),
  (2, 4),
  (3, 0),
  (3, 2),
  (3, 4),
  (3, 5),
  (4, 0),
  (5, 0),
  (5, 1),
  (5, 4),
  (5, 6),
  (4, 6)
]


xx = 1

We use integer identifiers for the nodes for efficiency. Note that, unlike this example, in a real web graph, not every page will have in-links, nor will every page have out-links.

## First Implementation and Test

\[10 points\] Implement the iterative PageRank algorithm. Test your code on the six-node example using the input representation given above.  Be sure that your code handles pages that have no in-links or out-links properly.  (You may wish to test on a few such examples.) In later parts of this assignment, depending on how you store the data, it may be convenient to use iterators rather than storing the data in memory.

In [13]:
# TODO: Implement PageRank, given nodes and edges, to start with a uniform
# distribution over nodes, run a fixed number of iterations, and
# return a distribution over nodes.


outlinks = {}

for node in small_nodes:
  if node[0] not in outlinks:
    outlinks[node[0]] = []

for item in small_edges:
  outlinks[item[0]].append(item[1])

print(outlinks)



def page_rank_fixed_iter(nodes, edges, iterations=10):
  P = len(nodes)
  I = [0] * P
  R = [0] * P

  for i in range(P):
    I[i] = 1 / P
  
  for i in range(iterations):
    for i in range(P):
      R[i] = 0.15 / P
    
    for node in nodes:
      Q = outlinks[node[0]]
      count = 0
      if len(Q):
        for q in Q:
          R[q] += (1 - 0.15) * I[node[0]] / len(Q)
      else:
        count += 1
        # for q in nodes:
        #   R[q[0]] += (1-0.15) * I[node[0]] / P 
    
      for q in nodes:
        R[q[0]] += count * ((1-0.15) * I[node[0]] / P)
    
    for i in range(P):
      I[i] = R[i]
  
  print(sum(R))
  return R
      

# Output PageRank on the toy graph at various points.
# Make sure your output has node number, name, and PageRank value.
print(page_rank_fixed_iter(small_nodes, small_edges, 1))
print(page_rank_fixed_iter(small_nodes, small_edges, 10))
print(page_rank_fixed_iter(small_nodes, small_edges, 100))

{0: [1, 2, 5], 1: [2, 3, 4, 5], 2: [3, 4], 3: [0, 2, 4, 5], 4: [0], 5: [0, 1, 4]}
1.0
[0.24930555555555556, 0.11944444444444444, 0.14305555555555555, 0.13124999999999998, 0.21388888888888885, 0.14305555555555555]
0.9999999999999998
[0.25203637602817186, 0.13930650918251075, 0.15129376593475066, 0.11896297277689877, 0.18710661014291707, 0.15129376593475066]
0.9999999999999999
[0.25212710537519556, 0.13930618531853856, 0.15130648986670525, 0.11890782257353917, 0.18704590699931606, 0.15130648986670525]


## PageRank on Web Crawl Data

\[20 points\] Download and unpack a list of `.edu` websites and the links among them from the [Common Crawl](https://commoncrawl.org/2017/05/hostgraph-2017-feb-mar-apr-crawls/) open-source web crawl. For the sake of brevity, the data record links among websites, not web pages. The information for nodes and links is the same as the toy example above.

In [ ]:
# If you're running on a machine (e.g., Windows) that doesn't have wget or gzip,
# feel free to comment this out and use a different set of commands to load
# the data.
!wget https://ccs.neu.edu/home/dasmith/courses/cs6200/vertices-edu.txt.gz
!gzip -df vertices-edu.txt.gz
!wget https://ccs.neu.edu/home/dasmith/courses/cs6200/edges-edu.txt.gz
!gzip -df edges-edu.txt.gz

There should now be files `vertices-edu.txt` and `edges-edu.txt`.

In [ ]:
# TODO: Process the raw data into the same format as the simple graph.
# You may create lists or iterators.

Refine your implementation of PageRank to test for numerical convergence. Specificially, at each iteration, calculate the [perplexity](https://en.wikipedia.org/wiki/Perplexity) of the PageRank distribution, where perplexity is defined as 2 raised to the [Shannon entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory)) of the PageRank distribution, i.e., $2^{H(PR)}$. (Recall that we defined entropy when talking about data compression.) The maximum perplexity of a PageRank distribution will therefore be the number of nodes in the graph.

At each iteration, check the _change_ in perplexity. If the change is less than some threshold, you can stop.


In [ ]:
# TODO: Implement convergence testing in PageRank
# If you choose, you can share some subroutines with your first version.
# Print the change in perplexity at each iteration.

def page_rank(nodes, edges, threshold=1):
  return []

# Run until perplexity changes by less than 1
PR = page_rank(nodes, edges, 1)

## Link Analysis

\[20 points\] In this final section, you will compute some properties of the web-site graph you downloaded.

First, consider the _in-link count_ of a website, simply the number of web-sites pointing to it (including self-links). 

In [ ]:
# TODO: List the document ID, domain name, and in-link count of the 60 websites with the highest in-link count

Then, use the PageRank values compute by your second implementation. Note that some websites will have both a high in-link count and PageRank.

In [ ]:
# TODO: List the document ID, domain name, and PageRank of the 60 websites with the highest PageRank.

Finally, compute some summary statistics on this dataset.

In [ ]:
# TODO: Compute:
# - the proportion of websites with no in-links (i.e., source nodes);

# - the proportion of websites with no out-links (i.e., sink nodes);

# - the proportion of websites whose PageRank is higher than the initial uniform distribution.
